<a href="https://colab.research.google.com/github/vanilsongomes/Pratical/blob/master/Prophet_Covid19_Breves_FAMAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://media.giphy.com/media/dVuyBgq2z5gVBkFtDc/giphy.gif)

# **FACULDADE DE MATEMÁTICA - UFPA/CUMB**
## **PREVISÃO DO COVID-19 EM BREVES-PA**

### <center>Analysis of the COVID-19 Pandemic Machine Learning Technics for Forecasting</center>

## Coronavirus (COVID-19) 

O coronavírus é uma família de vírus com o nome da coroa pontiaguda. O novo coronavírus, 
também conhecido como SARS-CoV-2, é um vírus respiratório contagioso que foi relatado pela 
primeira vez em Wuhan, na China. Em 11/2/2020, a Organização Mundial de Saúde designou o 
nome COVID(COrona VIrus Disease)-19 para a doença causada pelo novo coronavírus. Este notebook tem como objetivo 
explorar o COVID-19 através de análises e projeções de dados.


In [ ]:
################# importa bibliotecas para o projeto ######################
import numpy as np 
import pandas as pd ## bibliotecas para preparação dos dataset
######################### bibliotecas gráficas##############################
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
######################## Prediction Prophet ##################################
from fbprophet import Prophet
import warnings
warnings.filterwarnings('ignore')
###################### Gráficos iterativos ##################################
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_rows',20000, 'display.max_columns',100)
#############################################################################
from datetime import datetime, timedelta
import datetime
# template para gráfico 
plt.style.use('fivethirtyeight')
%matplotlib inline

### 1 - PROCESSAMENTO DOS DADOS

In [ ]:
###CARREGA MANUALMENTE
##df_breves=pd.read_excel('Atual_Dados_Breves_COVID19.xlsx')
#df_breves.head(3)

In [ ]:
url = "https://github.com/vanilsongomes/Pratical/blob/master/Atual_Dados_Breves_COVID19.xlsx?raw=true"
df_breves = pd.read_excel(url,sheet_name='Planilha1')
df_breves.tail()

,Date,Daily Cases,Confirmed,Deaths,Recovered,Discarded,Analyze
98,2020-07-09,15,1392,73,1038,850,5
99,2020-07-10,18,1410,74,1055,855,5
100,2020-07-11,11,1424,74,1073,862,2
101,2020-07-12,11,1435,74,1085,870,4
102,2020-07-13,10,1445,74,1100,877,4


In [ ]:
##BuGn
##Wistia
df_breves.tail(14).style.background_gradient(cmap='BuGn')

,Date,Daily Cases,Confirmed,Deaths,Recovered,Discarded,Analyze
89,2020-06-30 00:00:00,21,1251,73,885,778,2
90,2020-07-01 00:00:00,22,1273,73,906,785,2
91,2020-07-02 00:00:00,22,1295,73,926,798,2
92,2020-07-03 00:00:00,23,1318,73,945,810,3
93,2020-07-04 00:00:00,18,1336,73,968,818,3
94,2020-07-05 00:00:00,11,1347,73,986,823,2
95,2020-07-06 00:00:00,8,1355,73,1002,818,2
96,2020-07-07 00:00:00,9,1364,73,1014,835,2
97,2020-07-08 00:00:00,13,1377,73,1025,842,5
98,2020-07-09 00:00:00,15,1392,73,1038,850,5


* Creating new feature "Active" 
* Active = Confirmed - Deaths - Recovered

In [ ]:
#Total cases of carona in Breves
###df_breves['Total Cases'] = df_breves['Recovered'] + df_breves['Deaths'] + df_breves['Confirmed']
###Active cases of carona in Breves
df_breves['Active Cases'] = df_breves['Confirmed'] - df_breves['Recovered'] - df_breves['Deaths']
df_breves.tail().style.background_gradient(cmap='Blues')

,Date,Daily Cases,Confirmed,Deaths,Recovered,Discarded,Analyze,Active Cases
98,2020-07-09 00:00:00,15,1392,73,1038,850,5,281
99,2020-07-10 00:00:00,18,1410,74,1055,855,5,281
100,2020-07-11 00:00:00,11,1424,74,1073,862,2,277
101,2020-07-12 00:00:00,11,1435,74,1085,870,4,276
102,2020-07-13 00:00:00,10,1445,74,1100,877,4,271


### 2 - VISUALIZAÇÃO GRÁFICA 

In [ ]:
carona_data = df_breves.groupby(['Date'])['Active Cases','Recovered'].sum().reset_index()
labels = ['Active Cases','Recovered']
values = [carona_data['Active Cases'].iloc[-1], carona_data['Recovered'].iloc[-1]]
# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value',textfont_size=15,
                  marker=dict(colors=['#ff3300','#00ff00'], line=dict(color='#FFFFFF', width=2.5)))
fig.update_layout(
    title='COVID-19 ACTIVE CASES VS RECOVERED')
fig.show()

In [ ]:
#ax = df_breves['Confirmed'].diff(1).plot()
#df_breves['Confirmed'].diff(1).rolling(7).mean().plot(ax=ax, label='7 Day Average')
#carona_data['Daily Cases_mean'] = carona_data['Confirmed'].diff(1).rolling(7).mean()
#Daily Cases in Breves Datewise
carona_data = df_breves.groupby(['Date'])['Confirmed'].sum().reset_index().sort_values('Confirmed',ascending = True)
carona_data['Daily Cases'] = carona_data['Confirmed'].sub(carona_data['Confirmed'].shift())
carona_data['Daily Cases'].iloc[0] = carona_data['Confirmed'].iloc[0]
carona_data['Daily Cases'] = carona_data['Daily Cases'].astype(int)
fig = px.bar(carona_data, y='Daily Cases', x='Date',hover_data =['Daily Cases'], color='Daily Cases', height=500)
fig.update_layout(
    title='Daily Cases in Breves Datewise')
fig.show()

In [ ]:
carona_data['Corona Growth Rate'] = carona_data['Confirmed'].pct_change().mul(100).round(2)
#Corona Growth Rate Comparison with Previous Day
fig = px.bar(carona_data, y='Corona Growth Rate', x='Date',hover_data =['Corona Growth Rate','Confirmed'], height=600)
fig.update_layout(
    title='Corona Growth Rate(in Percentage) Comparison with Previous Day')
fig.show()

In [ ]:
#Moratality Rate
carona_data = df_breves.groupby(['Date'])['Confirmed','Active Cases','Deaths'].sum().reset_index().sort_values('Date',ascending=False)

carona_data['Mortality Rate'] = ((carona_data['Deaths']/carona_data['Confirmed'])*100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=carona_data['Date'], y=carona_data['Mortality Rate'],
                    mode='lines+markers',name='Cases',marker_color='red'))
fig.update_layout(title_text='COVID-19 Mortality Rate in BREVES',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [ ]:
#DAILY DEATHS IN BREVES
carona_data = df_breves.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Deaths',ascending = True)
carona_data['DAILY DEATHS'] = carona_data['Deaths'].sub(carona_data['Deaths'].shift())
carona_data['DAILY DEATHS'].iloc[0] = carona_data['Deaths'].iloc[0]
carona_data['DAILY DEATHS'] = carona_data['DAILY DEATHS'].astype(int)
fig = px.bar(carona_data, y='DAILY DEATHS', x='Date',hover_data =['DAILY DEATHS'], color='DAILY DEATHS', height=500)
fig.update_layout(
    title='DAILY DEATHS IN BREVES')
fig.show()

In [ ]:
#Recovery Rate
carona_data = df_breves.groupby(['Date'])['Confirmed','Active Cases','Recovered'].sum().reset_index().sort_values('Date',ascending=False)
carona_data['Recovery Rate'] = ((carona_data['Recovered']/carona_data['Confirmed'])*100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=carona_data['Date'], y=carona_data['Recovery Rate'],
                    mode='lines+markers',name='Cases',marker_color='green'))
fig.update_layout(title_text='COVID-19 Recovery Rate in BREVES',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [ ]:
#DAILY Recovery IN BREVES
carona_data =  df_breves.groupby(['Date'])['Recovered'].sum().reset_index().sort_values('Recovered',ascending = True)
carona_data['DAILY RECOVERY'] = carona_data['Recovered'].sub(carona_data['Recovered'].shift())
carona_data['DAILY RECOVERY'].iloc[0] = carona_data['Recovered'].iloc[0]
carona_data['DAILY RECOVERY'] = carona_data['DAILY RECOVERY'].astype(int)
fig = px.bar(carona_data, y='DAILY RECOVERY', x='Date',hover_data =['DAILY RECOVERY'], color='DAILY RECOVERY', height=500)
fig.update_layout(
    title='DAILY RECOVERY IN BREVES')
fig.show()

In [ ]:
#Total Cases,Active Cases,Cured,Deaths from Corona Virus in Breves
carona_data = df_breves.groupby(['Date'])['Confirmed','Active Cases','Recovered','Deaths'].sum().reset_index().sort_values('Date',ascending=False)
fig = go.Figure()
fig.add_trace(go.Scatter(x=carona_data['Date'], y=carona_data['Confirmed'],
                    mode='lines+markers',name='Confirmed'))
fig.add_trace(go.Scatter(x=carona_data['Date'], y=carona_data['Active Cases'], 
                mode='lines+markers',name='Active Cases'))
fig.add_trace(go.Scatter(x=carona_data['Date'], y=carona_data['Recovered'], 
                mode='lines+markers',name='Recovered'))
fig.add_trace(go.Scatter(x=carona_data['Date'], y=carona_data['Deaths'], 
                mode='lines+markers',name='Deaths'))
fig.update_layout(title_text='Curve Showing Different Cases from COVID-19 in Breves',plot_bgcolor='rgb(225,230,255)')
fig.show()

### 3 - Predicting the future with Facebook Prophet

Existem alguns métodos diferentes para prever tendências futuras, por exemplo [ARIMA](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average), [ARCH](https://en.wikipedia.org/wiki/Autoregressive_conditional_heteroskedasticity), [regressive models](https://en.wikipedia.org/wiki/Autoregressive_model), [neural networks](https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589).

Neste trabalho, veremos o [Prophet] (https://facebook.github.io/prophet/), uma biblioteca para previsão de séries temporais lançada pelo Facebook e de código aberto em 23 de fevereiro de 2017. Também tentaremos no problema de prever o número de casos confirmados e mortes por COVID-19 em Breves/Pa.


Agora, vamos dar uma olhada em como o Prophet funciona. Em sua essência, esta biblioteca utiliza o [additive regression model](https://en.wikipedia.org/wiki/Additive_model) $y(t)$ compreendendo os seguintes componentes:

$$y(t) = g(t) + s(t) + h(t) + \epsilon_{t},$$


onde:
* Tendência $g(t)$ mudanças não periódica.
* Sazonalidade $s(t)$ representa a mudança periódica.
* Components de feriados $h(t)$ contribui com informações sobre feriados e eventos.

Below, we will consider some important properties of these model components.

### Trend

The Prophet library implements two possible trend models for $g(t)$.

The first one is called *Nonlinear, Saturating Growth*. It is represented in the form of the [logistic growth model](https://en.wikipedia.org/wiki/Logistic_function):

$$g(t) = \frac{C}{1+e^{-k(t - m)}},$$

where:
* $C$ is the carrying capacity (that is the curve's maximum value).
* $k$ is the growth rate (which represents "the steepness" of the curve).
* $m$ is an offset parameter.

This logistic equation allows modelling non-linear growth with saturation, that is when the growth rate of a value decreases with its growth. One of the typical examples would be representing the growth of the audience of an application or a website.

Actually, $C$ and $k$ are not necessarily constants and may vary over time. Prophet supports both automatic and manual tuning of their variability. The library can itself choose optimal points of trend changes by fitting the supplied historical data. 

Also, Prophet allows analysts to manually set changepoints of the growth rate and capacity values at different points in time. For instance, analysts may have insights about dates of past releases that prominently influenced some key product indicators.

The second trend model is a simple *Piecewise Linear Model* with a constant rate of growth. It is best suited for problems without saturating growth.

### Seasonality

The seasonal component $s(t)$ provides a flexible model of periodic changes due to weekly and yearly seasonality.

Weekly seasonal data is modeled with dummy variables. Six new variables are added: `monday`, `tuesday`, `wednesday`, `thursday`, `friday`, `saturday`, which take values 0 or 1 depending on the day of the week. The feature `sunday` is not added because it would be a linear combination of the other days of the week, and this fact would have an adverse effect on the model.

Yearly seasonality model in Prophet relies on Fourier series.

Since [version 0.2](https://github.com/facebook/prophet) you can also use *sub-daily time series* and make *sub-daily forecasts* as well as employ the new *daily seasonality* feature.

### Holidays and Events

The component $h(t)$ represents predictable abnormal days of the year including those on irregular schedules, e.g., Black Fridays.

To utilize this feature, the analyst needs to provide a custom list of events.

### Error

The error term $\epsilon(t)$ represents information that was not reflected in the model. Usually it is modeled as normally distributed noise.

### 4 - SIMULAÇÃO PROPHET

In [ ]:
#Forecasting of Total Cases for Next 30 Days
df = df_breves.groupby('Date')['Confirmed'].sum().reset_index()
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df['ds'])
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'],
                    mode='lines+markers',name='Predicted Cases',marker_color='Black'))
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'],
                    mode='lines+markers',name='Actual Cases',marker_color='red'))
fig.update_layout(
    title='Forecasting of Confirmed Cases in BREVES for Next 30 Days')
fig.show()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
#Forecasting of Deaths for Next 30 Days
df =df_breves.groupby('Date')['Deaths'].sum().reset_index()
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df['ds'])
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'],
                    mode='lines+markers',name='Predicted Cases',marker_color='red'))
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'],
                    mode='lines+markers',name='Actual Cases',marker_color='blue'))
fig.update_layout(
    title='Forecasting of Deaths in BREVES for Next 30 Days')
fig.show()


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
#Forecasting of Recovered  for Next 30 Days
df = df_breves.groupby('Date')['Recovered'].sum().reset_index()
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df['ds'])
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'],
                    mode='lines+markers',name='Predicted Cases',marker_color='green'))
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'],
                    mode='lines+markers',name='Actual Cases',marker_color='yellow'))
fig.update_layout(
    title='Forecasting of Recovered in BREVES for Next 30 Days')
fig.show()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


# Referências
[COVID-19 BREVES](https://sites.google.com/view/coronavirusbreves/)

[Painel Coronavírus](https://covid.saude.gov.br/)

[Vigilância Epidemiológica - SESPA](https://www.covid-19.pa.gov.br/#/)

[Prophet](https://facebook.github.io/prophet/)

[Forecasting at Scale](https://peerj.com/preprints/3190.pdf)

[Prophet: forecasting at scale](https://research.fb.com/blog/2017/02/prophet-forecasting-at-scale/)